In [2]:
import json
import os
import re
import time

import numpy as np
import pandas as pd

from collections import defaultdict
from geopy.geocoders import Nominatim

%matplotlib inline
pd.options.display.max_columns = 99
os.chdir("/Users/yenchenchou/Documents/RMDS_YC/RiskScore/RMDS_COVID19_riskgenerator")

In [2]:
df = pd.read_csv(
    "data/external/Core-USA-July2020-Release-CORE_POI-2020_06-2020-07-13/core_poi-part1.csv.gz",
    compression="gzip")

In [3]:
df.head(5)

,safegraph_place_id,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,iso_country_code,phone_number,open_hours,category_tags,opened_on,closed_on,tracking_opened_since,tracking_closed_since
0,sg:0026d587c9b145b58272cbac7bdea56a,NaN,Vallone A John DDS,NaN,NaN,Offices of Dentists,Offices of Dentists,621210.0,41.158062,-87.891611,35 Briarcliff Prof Ctr,Bourbonnais,IL,60914,US,1.815939e+10,NaN,NaN,NaN,NaN,NaN,2019-07
1,sg:0037c4dc85f642c6aeaaf318e41b1bf7,NaN,Mitchell Cosmetology College,NaN,NaN,"Colleges, Universities, and Professional Schools","Colleges, Universities, and Professional Schools",611310.0,33.243518,-86.816563,116 1st St S,Alabaster,AL,35007,US,1.205664e+10,"{ ""Mon"": [], ""Tue"": [[""12:00"", ""16:00""]], ""Wed...",NaN,NaN,NaN,NaN,2019-07
2,sg:0041bdf50f984826acd68ce19922ecd4,NaN,Sullivan Chris M Dr,NaN,NaN,"Other Professional, Scientific, and Technical ...",Veterinary Services,541940.0,32.153705,-91.715387,403 Eighth St,Winnsboro,LA,71295,US,NaN,NaN,NaN,NaN,NaN,NaN,2019-07
3,sg:005761d9681947629bf02a64cfd4aec2,NaN,U Pick Carlsbad Strawberry Co,NaN,NaN,Specialty Food Stores,Fruit and Vegetable Markets,445230.0,33.135632,-117.326437,1000 Cannon Rd,Carlsbad,CA,92008,US,NaN,"{ ""Mon"": [[""9:00"", ""18:00""]], ""Tue"": [[""9:00"",...",NaN,NaN,NaN,NaN,2019-07
4,sg:007236f9d77d4158bbd4d55f39d98e49,NaN,CJ's Auto Glass Repair,NaN,NaN,Lumber and Other Construction Materials Mercha...,Other Construction Material Merchant Wholesalers,423390.0,40.470182,-74.408106,190 State Route 18,East Brunswick,NJ,8816,US,1.732994e+10,"{ ""Mon"": [[""8:00"", ""20:00""]], ""Tue"": [[""8:00"",...",NaN,NaN,NaN,NaN,2019-07


In [4]:
df.shape

(1179095, 22)

In [5]:
def data_filter(df):
    
    filter_ls = ["Restaurants and Other Eating Places", "Grocery Stores"]
    #     df = df[df["top_category"].isin(filter_ls)].copy()
    
    df = df[df["region"] == "CA"].copy()
    df = df[["safegraph_place_id", 
            "latitude",
            "longitude",
            "city",
            "postal_code"]]
    df["community"] = df["postal_code"].apply(lambda x: mapping(x))
    df = df[df["community"].notnull()]
    
    return df


def mapping(x):
    with open("data/internal/zipcode_mapper.json") as file:
        mapper_dict = json.load(file)
    mapper_dict = {str(key):val for key, val in mapper_dict.items()}
    try: 
        return mapper_dict[str(x)]
    except:
        pass

In [6]:
now = time.time()
df_filter = data_filter(df)
later = time.time()
print(later - now)

41.280043601989746
